## Ejemplo 4: Agrupando datos con `groupby`

### 1. Objetivos:
    - Aprender a usar `grouby` para segmentar nuestros conjuntos de datos y aplicar funciones agregadoras a cada segmento.
 
---
    
### 2. Desarrollo:

#### a) Segmentando datos con `groupby`

En nuestro Reto pasado construimos un nuevo conjunto de datos agregando la información de las tablas `occupations` y `age_ranges` a la tabla `users`. Vamos a leer este dataset primero:

In [1]:
import pandas as pd

In [3]:
users = pd.read_csv('../../Datasets/MovieLens/users-full.csv', index_col=0)

users

,gender,age_id,age_range,occupation_id,occupation,cp
user_id,,,,,,
1,F,1,Under 18,10,K-12 student,48067
2,M,56,56+,16,self-employed,70072
3,M,25,25-34,15,scientist,55117
4,M,45,45-49,7,executive/managerial,02460
5,M,25,25-34,20,writer,55455
...,...,...,...,...,...,...
6036,F,25,25-34,15,scientist,32603
6037,F,45,45-49,1,academic/educator,76006
6038,F,56,56+,1,academic/educator,14706


Vamos a ver qué pasa si agrupamos nuestro conjunto usando la columna `gender`:

In [4]:
users.groupby('gender')

Para ver algún resultado más legible tenemos que aplicar funciones agregadoras a nuestro objeto `groupby`:

In [5]:
users.groupby('gender').size()

gender
F    1709
M    4331
dtype: int64

`size` nos hace un conteo de cuántas muestras hay en cada grupo y regresa el total. Ahora podemos ver entonces que hay 1709 mujeres y 4331 hombres en nuestro dataset.

También podemos pedir columnas específicas de nuestros grupos y aplicar agregaciones a cada columna:

In [6]:
users.groupby('gender')['occupation'].value_counts()

gender  occupation            
F       college/grad student      234
        other or not specified    232
        academic/educator         209
        executive/managerial      139
        doctor/health care        102
        clerical/admin            100
        artist                     91
        homemaker                  89
        sales/marketing            79
        writer                     78
        K-12 student               66
        technician/engineer        52
        self-employed              51
        programmer                 50
        retired                    34
        customer service           31
        scientist                  28
        lawyer                     22
        unemployed                 15
        tradesman/craftsman         4
        farmer                      3
M       executive/managerial      540
        college/grad student      525
        other or not specified    479
        technician/engineer       450
        programmer 

Podemos usar dos o más columnas para agrupar también. Lo que sucede es que el dataset se agrupa usando la primer columna, y luego, dentro de cada grupo se hace una segunda agrupación usando la segunda columna:

In [7]:
users.groupby(['gender', 'age_range'])['occupation'].value_counts()

gender  age_range  occupation            
F       18-24      college/grad student      163
                   other or not specified     32
                   academic/educator          18
                   sales/marketing            15
                   writer                     14
                                            ... 
M       Under 18   executive/managerial        1
                   farmer                      1
                   lawyer                      1
                   programmer                  1
                   retired                     1
Name: occupation, Length: 241, dtype: int64

Aquí hemos segmentado nuestro dataset en dos niveles. En el primer nivel, podemos obtener datasets independientes para cada género:

In [8]:
users_ga_counts = users.groupby(['gender', 'age_range'])['occupation'].value_counts()

In [9]:
users_ga_counts.loc['F']

age_range  occupation            
18-24      college/grad student      163
           other or not specified     32
           academic/educator          18
           sales/marketing            15
           writer                     14
                                    ... 
Under 18   other or not specified      9
           artist                      2
           unemployed                  2
           academic/educator           1
           executive/managerial        1
Name: occupation, Length: 110, dtype: int64

En un segundo nivel, podemos obtener datasets por cada rango de edades en cada género:

In [10]:
users_ga_counts.loc[('F', '18-24')]

occupation
college/grad student      163
other or not specified     32
academic/educator          18
sales/marketing            15
writer                     14
artist                      9
clerical/admin              9
technician/engineer         6
unemployed                  6
customer service            5
homemaker                   5
K-12 student                3
doctor/health care          3
executive/managerial        3
programmer                  3
self-employed               2
lawyer                      1
scientist                   1
Name: occupation, dtype: int64

¡Genial!

Ahora, no todas las funciones están disponibles "out-of-the-box" para ser aplicadas a objetos `groupby`. Hay algunas funciones que no podemos utilizar directamente y que para poder aplicarlas necesitamos usar el método `agg`. `agg` recibe una función o una lista de funciones y se las aplica a las columnas solicitadas de cada grupo.

Por ejemplo, podemos encontrar la "moda" (la categoría que más veces aparece en una columna específica) de esta manera:

In [11]:
users.groupby('gender')['occupation'].agg(pd.Series.mode)

gender
F    college/grad student
M    executive/managerial
Name: occupation, dtype: object

Podemos aplicar la función a dos columnas al mismo tiempo:

In [13]:
users.groupby('gender')[['age_range', 'occupation']].agg(pd.Series.mode)

,age_range,occupation
gender,,
F,25-34,college/grad student
M,25-34,executive/managerial


Y también podemos aplicar varias funciones al mismo tiempo pasándole a `agg` una lista de funciones. En este caso vamos a usar algunos análisis estadísticos a la columna `age_id`. En realidad estos análisis no van a ser precisos porque esta columna contiene ids que representan rangos de edades, no edades como tal. Pero considéralo un simple ejemplo para ver cómo funcionan las herramientas:

In [15]:
users.groupby('gender')['age_id'].agg(['mean', 'median', 'std'])

,mean,median,std
gender,,,
F,30.859567,25,13.242564
M,30.552297,25,12.757110


Vamos ahora a practicar nuestras nuevas herramientas en unos cuantos Retos.